# POS Tagging

## Intro: What is a word?
* A Word: Smallest semantic unit
* Morphology: structure of sentences made up of words
* Morpheme: smallest meaningful unit of a word, such as
    * Stems: The core of the word
    * Affixes: AddOns such as "un"-attractive, run-"s"
    

## Word Classes aka Parts of Speech (PoS)
* Tell us about neighbouring words
* Gives syntactic structure to language
* Useul for information extraction and speech recognition

Examples:
* Noun
* Verb
* Adj
* Adv: degree of something ex "slowly", "luckily"
* Preposition
* Pronoun: I, me, mine, he, her
* Determiner: The, a, that, those

### Open and Closed classes
Closed - established:
* Prepositions
* Pronouns
* Conjunctions
* Particles

Open - constantly growing:
* Nouns
* Verbs
* Adjectives
* Adverbs

### Ambiguity
Some words can belong to different word classes depending on the context, ex 
* The dogs bite the cat

Bite can be either verb or noun (here verb)

### Penn-Treebank
Complete database and tagset

## HMM PoS Tagger
Let a HMM be defined by following parameters

$$HMM = (Q, O, A, B)$$

* States: $Q = q_1 ... q_N$ [parts of speech]
* Observations: $O = o_1 ... o_V$ [words]
* Transition matrix: $A = a_{ij} = P(y_t = q_j \vert y_{t-1} = q_i)$
* Emission matrix: $B = b_{ik} = P(x_t = o_k \vert y_t = q_i)$

### Markov Chain
We modelling tagging words as PoS as a Hidden Markov Process

We rely on 2 indenpendence assumptions:

__Markov Independence__
$$P(q_i \vert q_1 ... q_{i-1}) \approx P(q_i \vert q_{i-1})$$
* Current state depends only on the previous state

__Output Independence__
$$P(o_i \vert q_1 ... q_i ... q_T, o_1 ... o_i ... o_T) = P(o_i \vert q_i)$$
* Current observation only depends on the state that created it

### Generative process
Likelihood: $$P(O \vert Q; \theta) = \prod_t^T P(q_i \vert q_{i-1}) P(o_i \vert q_i)$$
* $\theta = (A,B)$

### Inference with HMMs
* Given $O$ compute $P(O)$
* Given $O$ compute most likely sequence of POS $Q=q_1...q_N$
* $Q = q_i...q_N$ estimate params $\theta = (A,B)$

## Likelihood of Observation Sequence
### Observation Probability
Sum over sequence $Q$, use likelihood $P(O \vert Q; \theta)$.

$$P(O; \theta) = \sum_{q \in Q} P(O \vert Q; \theta)$$

Problem: $N^T$ possible hidden sequences for $N$ hidden states and $T$ sequences, this is infeasible for large values.

### Forward Algorithm - Compute Likelihood of Observed Seq
The problem can be solved with dynamic programming, i.e. storing intermediate results to avoid computing them over and over.

Use forward trellis $\alpha$, where $\alpha_t(j)$ is the probability of ending up in state $j$ after seeing the first $t$ observations given the automation $\lambda = (A, B)$

$$\alpha_t (j) = P(o_1...o_t, q_t=j \vert \lambda)$$

We compute $\alpha_t (j)$ by summing over the extensions of all paths leading to the current cell.

Let:
* $\alpha_{t-1} (i)$: The previous forward path prob from previous time step
* $a_{ij} = p(q_i | q_j)$: Transition probability from state $q_i$ to $q_j$
* $b_j(o_t) = p(o_t | q_j)$: Likelihood of observing $o_t$ given current state $j$.


### Deriving Trellis
Deriving an expression for $\alpha$ which allows dynamic programming using the chain rule:

$\alpha^t (j) = P(o^{1:t}, q^t = q_j)$
* $=\sum_{i=1}^N p(o^{1:t-1}, o^{t}, q^t = q_j, q^{t-1} = q_i)$

* $=\sum_{i=1}^N p(o^{t} | o^{1:t-1}, q^t = q_j, q^{t-1} = q_i) \cdot p(o^{1:t-1}, q^t = q_j, q^{t-1} = q_i)$

* $=\sum_{i=1}^N p(o^{t} | o^{1:t-1}, q^t = q_j, q^{t-1} = q_i) \cdot p(q^t = q_j | o^{1:t-1}, q^{t-1} = q_i) \cdot p(o^{1:t-1}, q^{t-1} = q_i)$

Since the current state only depends on the previous state and an obs. only depends on the corresponding hidden state, we can re-write a couple of terms:

* $p(o^{t} | o^{1:t-1}, q^t = q_j, q^{t-1} = q_i) =  p(o^{t} | q^t = q_j)  = b_j(o^t)$ [output indep.]

* $p(q^t = q_j | o^{1:t-1}, q^{t-1} = q_i) =  p(q^t = q_j | q^{t-1} = q_i) = a_{ij}$ [markov indep. and def of A]

By definition of $\alpha$, we can re-write the following term to be the previous iteration.
* $p(o^{1:t-1}, q^{t-1} = q_i) = \alpha^{t-1}$

Using the 3 terms again we achieve:
$$\alpha^t(j) = \sum_{i=1}^N b_j(o^t) \cdot a_{ij} \cdot \alpha^{t-1}$$

#### Forward Algorithm Steps
The algorithm uses 3 steps:
1. Init step
$$\alpha_1 = \pi_j b_j(o_1) \quad 1\leq j \leq N$$

2. Recursion step
$$\alpha_t (j) = \sum_{i=1}^N \alpha_{t-1} (i) a_{ij} b_j(o_t) \quad 1\leq j \leq N, 1 < t \leq T$$

3. Termination step (sum over alpha for last timestep)
$$P(O \vert \lambda) = \sum_{i=1}^N \alpha_T (i)$$

### Decoding
__Decoding__
* Given $\lambda = (A,M)$ and $O= o_1 ... o_T$ find the most probably sequence of hidden states $Q = q_1 ... q_T$

Example: 
* Weather is the hidden variable, and number of ice creams eaten on a day is the observed variable.
* __Decoding__ is then to find the most likely weather sequence - given the ice cream sequence.

### Viterbi algorithm - Compute Most Probable Hidden Seq
Algorithm for decoding
* Also makes use of a dynamic programming trellis.
* Process the observed sequence left to right, in order to fill out trellis

$$v_t(j) = max_i \quad v_{t-1} (i) a_{ij} b_j(o_t) \quad i=1...N$$

The trellis entry $v_t(j)$ is the prob. of HMM being in state $j$ after seeing the first $t$ observations, where it has passed through the most probable sequence $q_1 ...q_{t-1}$.

* v_{t-1}(i): Previous viterbi path probability from prev step
* $a_{ij}$: Transition probability from prev. state $q_i$ to $q_j$
* $b_j(o_t)$: State observation likelihood for $o_t$ given current state $j$

Similar to the forward algorithm, except for:
* Uses the __max__ over previous path probabilities, forward algo uses the __sum__.
* Backpointers: In order to produce an observation likelihood we have to keep track of the path which lead to each state.


Algorithm steps:

1. Init
 * Init trellis: $v_1(j) = \pi_j b_j(o_1)$
 * Init emission matrix: $b_{t_1}(j) = 0 $
 * Where $1 \leq j \leq N$
 
2. Recursion
 * Update trellis: $v_t(j) = max \ v_{t-1}(i) a_{ij} b_j(o_t)$
 * Update emission matrix: $b_{t_t}(j) = argmax_i \ v_{t-1} (i) a_{ij} b_j(o_t)$
 * Where $1 \leq j \leq N$ and $1 < t \leq T$
 
3. Termination
 * Highest probability: $P^* = max \ v_T (i)$
 * Backtrace: $q_T^* = argmax \ v_T(i)$

### Forward-Backward Algorithm - Learn Params
Goal: Learn $\lambda = (A,B)$ given $O$ and a set of _possible_ states in the HMM, $Q$. This is done through expectation maximization.

Done iteratively through Baum-Welch, EM algorithm.

For PoS Tagging this is actually unnecessary since our hidden states are in fact observed! We have PoS tags for our words through the Penn Tree Bank

#### Example
Toy example for ice cream: 
* $Q = \{q_1, q_2\} = \{hot, cold\}$
* $O = [o_1, o_2, o_3] = [1, 2, 3]$

If given observation sequences matched with hidden state sequences then we can compute:

* $\pi_{q_1}$ and $\pi_{q_2}$ by simply counting how many start in the two state
* Compute A from the transitions, ex.
    * $p(q_1|q_1)$
    * $p(q_1|q_2)$
    * $p(q_2|q_2)$
    * $p(q_2|q_1)$
* Compute B matrix:
    * $p(o_1 | q_1)$, $p(o_1 | q_2)$
    * $p(o_2 | q_1)$, $p(o_2 | q_2)$
    * $p(o_3 | q_1)$, $p(o_3 | q_2)$

However we do not known the counts for being in any of the hidden states.

## Information Extraction
Who did what to whom and why?

### Named Entity Recognition
Task: Detecting spans of text which constitute a named entity in the text

Named entity: Person/location/organisation which is named
* Can be extended to entities such as prices, dates, times

### Sequence Labeling



